In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from itertools import combinations
import os
from seismic_purifier import (
    RepresentationLearningSingleAutoencoder, 
    RepresentationLearningDenoisingSingleAutoencoder, 
    RepresentationLearningMultipleAutoencoder
)

# -------------------------------
# Configuration
# -------------------------------

# Number of samples to plot
NUM_SAMPLES = 10

# Data paths - MODIFY THESE TO YOUR .npy FILES
WAVEFORM_DATA_PATH = "/home/ege/rawcovar_data/2020-09-09/processed_data/GAZK.npy"
LABELS_DATA_PATH = "/home/ege/rawcovar_data/2020-09-09/processed_data/GAZK_y_condensed.npy"

# Model configuration - MODIFY THESE
MODEL_WEIGHTS_PATH = "/home/ege/rawcovar/experiments/JUNE2025/29JUNE2025/1DAY_CONTINUOUS_EXP_2_2020-09/models/GAZK_representation_multiple_5epochs.h5"
MODEL_CLASS = RepresentationLearningMultipleAutoencoder  

# Choose from:
# - RepresentationLearningSingleAutoencoder
# - RepresentationLearningDenoisingSingleAutoencoder  
# - RepresentationLearningMultipleAutoencoder

# -------------------------------
# Core Functions
# -------------------------------

def compute_covariance(data_arrays):
    """
    Computes autocovariance or cross-covariance between signals.
    
    Args:
        data_arrays (np.ndarray): Variable number of 2D arrays, each with shape (timesteps, channels)
    
    Returns:
        lags (np.ndarray): Lag values
        avg_cov (np.ndarray): Averaged covariance
    """
    if len(np.shape(data_arrays)) == 2:
        data_arrays = np.expand_dims(data_arrays, axis=0)
    
    num_signals = len(data_arrays)    
    num_timesteps, num_channels = data_arrays[0].shape

    covariances = []
    lags = np.arange(-num_timesteps + 1, num_timesteps)

    if num_signals == 1:
        # Autocovariance
        data = data_arrays[0]
        for c in range(num_channels):
            channel_data = data[:, c]
            channel_data = channel_data - np.mean(channel_data)  # Zero-mean
            cov = np.correlate(channel_data, channel_data, mode='full')
            covariances.append(cov)
    else:
        # Cross-covariance between all possible pairs
        pairs = list(combinations(range(num_signals), 2))
        for idx1, idx2 in pairs:
            data1 = data_arrays[idx1]
            data2 = data_arrays[idx2]
            for c in range(num_channels):
                channel_data1 = data1[:, c]
                channel_data2 = data2[:, c]
                channel_data1 = channel_data1 - np.mean(channel_data1)  # Zero-mean
                channel_data2 = channel_data2 - np.mean(channel_data2)  # Zero-mean
                cov = np.correlate(channel_data1, channel_data2, mode='full')
                covariances.append(cov)

    covariances = np.array(covariances)
    avg_cov = np.mean(covariances, axis=0)
    return lags, avg_cov

def extract_features_from_model(waveforms, model_weights_path, model_class):
    """
    Load model weights and extract feature maps.
    
    Args:
        waveforms (np.ndarray): Shape (n_samples, 3000, 3)
        model_weights_path (str): Path to the .h5 weights file
        model_class: The model class from seismic_purifier
    
    Returns:
        feature_maps (np.ndarray): Extracted feature maps
    """
    print(f"Loading model weights from: {model_weights_path}")
    print(f"Using model: {model_class.__name__}")
    
    # Initialize the model
    model = model_class()
    model.compile()
    
    # Initialize model with dummy data
    dummy_input = np.random.normal(size=[1, 3000, 3])
    _ = model(dummy_input)
    
    # Load the trained weights
    model.load_weights(model_weights_path)
    print("Successfully loaded weights!")
    
    # Extract features
    print("Extracting features from waveforms...")
    feature_maps_list = []
    batch_size = 32
    
    for i in range(0, len(waveforms), batch_size):
        end_idx = min(i + batch_size, len(waveforms))
        batch = waveforms[i:end_idx]
        
        model_out = model(batch)
        
        if model_class == RepresentationLearningMultipleAutoencoder:
            # Extract first 5 feature maps for multiple autoencoder
            batch_features = list(model_out)[0:5]
            batch_features = np.array(batch_features)
            # Transpose to (batch_size, num_feature_maps, timesteps, features)
            batch_features = np.transpose(batch_features, axes=[1, 0, 2, 3])
        else:
            # Extract first feature map for single/denoising autoencoder
            batch_features = list(model_out)[0:1]
            batch_features = np.array(batch_features)
            # Transpose to (batch_size, num_feature_maps, timesteps, features)
            batch_features = np.transpose(batch_features, axes=[1, 0, 2, 3])
        
        feature_maps_list.append(batch_features)
        
        if (i + batch_size) % (batch_size * 10) == 0:
            print(f"  Processed {i + batch_size}/{len(waveforms)} samples...")
    
    # Concatenate all batches
    feature_maps = np.concatenate(feature_maps_list, axis=0)
    print(f"Extracted feature maps shape: {feature_maps.shape}")
    
    return feature_maps

def plot_waveform_channel(ax, timesteps, waveform, channel_idx, ylim_min=None, ylim_max=None, 
                         color='blue', show_xticks=True):
    """
    Plots a single waveform channel on the given axes.
    """
    channels = ['E', 'N', 'Z']
    ax.plot(timesteps, waveform, color=color, linewidth=1)
    if channel_idx == 0:
        ax.set_title("Waveform", fontsize=14, pad=5, fontweight='bold')
    
    # Add channel label
    ax.set_ylabel(f'{channels[channel_idx]} Channel', fontsize=10)
    
    if ylim_min is not None and ylim_max is not None:
        ax.set_ylim(ymin=ylim_min, ymax=ylim_max)
    
    ax.tick_params(axis='y', labelsize=10)
    if show_xticks:
        ax.set_xlabel('Timesteps', fontsize=12)
        ax.tick_params(axis='x', labelsize=10)
    else:
        ax.set_xlabel('')
        ax.set_xticklabels([])
        ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    
    ax.grid(True, alpha=0.3)

def plot_heatmap(ax, heatmap, vmin=None, vmax=None, title=None):
    """
    Plots the heatmap on the given axes.
    """
    if vmin is not None and vmax is not None:
        cax = ax.imshow(heatmap, aspect='auto', cmap='magma', origin='lower', vmin=vmin, vmax=vmax)
    else:
        cax = ax.imshow(heatmap, aspect='auto', cmap='magma', origin='lower')
        
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_xlabel('Timesteps', fontsize=12)
    ax.set_ylabel('Features', fontsize=12)
    plt.colorbar(cax, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

def plot_autocovariance(ax, lags, autocov, ylim_min=None, ylim_max=None, title=None, 
                       ylabel='Autocovariance'):
    """
    Plots the autocovariance function on the given axes.
    """
    if ylim_min is not None and ylim_max is not None:
        ax.set_ylim(ymin=ylim_min, ymax=ylim_max)
        
    ax.plot(lags, autocov)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_xlabel('Lag', fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True, alpha=0.3)

def load_data():
    """
    Load waveform and label data from .npy files.
    
    Returns:
        waveforms (np.ndarray): Shape (n_samples, 3000, 3)
        labels (np.ndarray): Shape (n_samples,)
    """
    print(f"Loading waveform data from: {WAVEFORM_DATA_PATH}")
    waveforms = np.load(WAVEFORM_DATA_PATH)
    print(f"Waveforms shape: {waveforms.shape}")
    
    print(f"Loading labels from: {LABELS_DATA_PATH}")
    labels = np.load(LABELS_DATA_PATH)
    print(f"Labels shape: {labels.shape}")
    
    # Validate data shapes
    assert len(waveforms.shape) == 3, f"Waveforms should be 3D (samples, timesteps, channels), got shape {waveforms.shape}"
    assert waveforms.shape[1] == 3000, f"Expected 3000 timesteps, got {waveforms.shape[1]}"
    assert waveforms.shape[2] == 3, f"Expected 3 channels, got {waveforms.shape[2]}"
    assert len(labels.shape) == 1, f"Labels should be 1D, got shape {labels.shape}"
    assert waveforms.shape[0] == labels.shape[0], f"Number of waveforms ({waveforms.shape[0]}) != number of labels ({labels.shape[0]})"
    
    print(f"\nData summary:")
    print(f"Total samples: {len(waveforms)}")
    print(f"Earthquake samples: {np.sum(labels > 0.5)}")
    print(f"Noise samples: {np.sum(labels <= 0.5)}")
    
    return waveforms, labels

# -------------------------------
# Main Execution
# -------------------------------

def main():
    # Load data
    print("Loading data...")
    waveforms, labels = load_data()
    
    # Extract features from the trained model
    print("\nExtracting features from trained model...")
    feature_maps = extract_features_from_model(waveforms, MODEL_WEIGHTS_PATH, MODEL_CLASS)
    
    # Generate visualizations
    WAVEFORM_COLORS = ['blue', 'green', 'red']  # Colors for E, N, Z channels
    
    # Separate earthquake and noise indices
    earthquake_indices = np.where(labels > 0.5)[0]
    noise_indices = np.where(labels <= 0.5)[0]
    
    print(f"\nFound {len(earthquake_indices)} earthquake samples and {len(noise_indices)} noise samples")
    
    # Ensure equal number of earthquake and noise samples
    NUM_PLOTS = min(len(earthquake_indices), len(noise_indices), NUM_SAMPLES)
    print(f"Will generate {NUM_PLOTS} comparison plots")
    
    # Create output directory if it doesn't exist
    output_dir = "latent_space_plots"
    os.makedirs(output_dir, exist_ok=True)
    
    # Determine covariance title based on model type
    if MODEL_CLASS == RepresentationLearningMultipleAutoencoder:
        latent_covariance_title = 'Latent Representation\nCross-covariance Function'
        latent_covariance_ylabel = 'Mean Cross-covariance'
    else:
        latent_covariance_title = 'Latent Representation\nAutocovariance Function'
        latent_covariance_ylabel = 'Autocovariance'
    
    for plot_idx in range(NUM_PLOTS):
        eq_idx = earthquake_indices[plot_idx]
        noise_idx = noise_indices[plot_idx]
        
        print(f"Generating plot {plot_idx + 1}/{NUM_PLOTS} (EQ idx: {eq_idx}, Noise idx: {noise_idx})")
        
        # Extract earthquake data
        eq_waveform = waveforms[eq_idx]
        eq_feature_map = feature_maps[eq_idx]
        lags_waveform_eq, autocov_waveform_eq = compute_covariance(eq_waveform)
        lags_heatmap_eq, autocov_heatmap_eq = compute_covariance(eq_feature_map)
        
        # Extract noise data
        noise_waveform = waveforms[noise_idx]
        noise_feature_map = feature_maps[noise_idx]
        lags_waveform_noise, autocov_waveform_noise = compute_covariance(noise_waveform)
        lags_heatmap_noise, autocov_heatmap_noise = compute_covariance(noise_feature_map)
        
        # Create a figure with a 1x2 grid: left for earthquake, right for noise
        fig = plt.figure(figsize=(20, 10))
        main_gs = GridSpec(1, 2, figure=fig, wspace=0.3)
        
        # --- Earthquake Column ---
        eq_gs = main_gs[0, 0].subgridspec(2, 2, wspace=0.3, hspace=0.4)
        
        # Top-Left: Waveform Channels
        eq_waveform_gs = eq_gs[0, 0].subgridspec(eq_waveform.shape[1], 1, hspace=0.3)
        timesteps_eq = np.arange(eq_waveform.shape[0])
        
        for channel in range(eq_waveform.shape[1]):
            ax = fig.add_subplot(eq_waveform_gs[channel, 0])
            show_xticks = (channel == eq_waveform.shape[1] - 1)
            plot_waveform_channel(ax, timesteps_eq, eq_waveform[:, channel], channel, 
                                  color=WAVEFORM_COLORS[channel % len(WAVEFORM_COLORS)], 
                                  show_xticks=show_xticks)
        
        # Top-Right: Heatmap
        ax_heatmap_eq = fig.add_subplot(eq_gs[0, 1])
        plot_heatmap(ax_heatmap_eq, eq_feature_map[0].T, title="Latent Representation")
        
        # Bottom-Left: Autocovariance of Waveform
        ax_autocov_waveform_eq = fig.add_subplot(eq_gs[1, 0])
        plot_autocovariance(ax_autocov_waveform_eq, lags_waveform_eq, autocov_waveform_eq, 
                            title='Waveform\nAutocovariance Function')
        
        # Bottom-Right: Autocovariance of Heatmap
        ax_autocov_heatmap_eq = fig.add_subplot(eq_gs[1, 1])
        plot_autocovariance(ax_autocov_heatmap_eq, lags_heatmap_eq, autocov_heatmap_eq, 
                            title=latent_covariance_title,
                            ylabel=latent_covariance_ylabel)
        
        # --- Noise Column ---
        # Use earthquake feature map range for consistent scaling
        feature_map_max = np.max(eq_feature_map[0])
        feature_map_min = np.min(eq_feature_map[0])
        
        autocov_heatmap_max = np.max(autocov_heatmap_eq)
        autocov_heatmap_min = np.min(autocov_heatmap_eq)
        
        noise_gs = main_gs[0, 1].subgridspec(2, 2, wspace=0.3, hspace=0.4)
        
        # Top-Left: Waveform Channels
        noise_waveform_gs = noise_gs[0, 0].subgridspec(noise_waveform.shape[1], 1, hspace=0.3)
        timesteps_noise = np.arange(noise_waveform.shape[0])
        
        for channel in range(noise_waveform.shape[1]):
            ax = fig.add_subplot(noise_waveform_gs[channel, 0])
            show_xticks = (channel == noise_waveform.shape[1] - 1)
            plot_waveform_channel(ax, timesteps_noise, noise_waveform[:, channel], channel, 
                                  color=WAVEFORM_COLORS[channel % len(WAVEFORM_COLORS)], 
                                  show_xticks=show_xticks)
        
        # Top-Right: Heatmap (with consistent scaling)
        ax_heatmap_noise = fig.add_subplot(noise_gs[0, 1])
        plot_heatmap(ax_heatmap_noise, noise_feature_map[0].T, feature_map_min, feature_map_max, 
                     "Latent Representation")
        
        # Bottom-Left: Autocovariance of Waveform
        ax_autocov_waveform_noise = fig.add_subplot(noise_gs[1, 0])
        plot_autocovariance(ax_autocov_waveform_noise, 
                            lags_waveform_noise, 
                            autocov_waveform_noise, 
                            title='Waveform\nAutocovariance Function')
        
        # Bottom-Right: Autocovariance of Heatmap (with consistent scaling)
        ax_autocov_heatmap_noise = fig.add_subplot(noise_gs[1, 1])
        plot_autocovariance(ax_autocov_heatmap_noise, 
                            lags_heatmap_noise, 
                            autocov_heatmap_noise,
                            autocov_heatmap_min,
                            autocov_heatmap_max,
                            latent_covariance_title,
                            latent_covariance_ylabel)
        
        # --- Add Column Titles ---
        fig.text(0.30, 0.935, 'Earthquake Sample', ha='center', va='center', 
                 fontsize=20, fontweight='bold')
        fig.text(0.725, 0.935, 'Noise Sample', ha='center', va='center', 
                 fontsize=20, fontweight='bold')
        
        # Adjust overall layout and save the figure
        plt.tight_layout(rect=[0, 0.03, 1, 0.92])
        
        # Save with informative filename
        filename = f"{output_dir}/latent_plot_pair_{plot_idx + 1:03d}_eq{eq_idx}_noise{noise_idx}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free memory
        
        if (plot_idx + 1) % 5 == 0:
            print(f"  Saved {plot_idx + 1} plots so far...")
    
    print(f"\nAll plots saved to '{output_dir}' directory!")
    print(f"Generated {NUM_PLOTS} comparison plots.")

# -------------------------------
# Alternative: Save features for later use
# -------------------------------

def save_features_for_later():
    """
    Extract and save feature maps to a .npy file for future use.
    """
    # Load data
    waveforms, labels = load_data()
    
    # Extract features
    feature_maps = extract_features_from_model(waveforms, MODEL_WEIGHTS_PATH, MODEL_CLASS)
    
    # Save to file
    output_path = "extracted_feature_maps.npy"
    np.save(output_path, feature_maps)
    print(f"Saved feature maps to: {output_path}")
    
    return feature_maps

if __name__ == "__main__":
    # Option 1: Run the full analysis
    main()
    
    # Option 2: Just extract and save features
    # save_features_for_later()

2025-06-29 18:53:04.010665: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-29 18:53:04.067716: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-29 18:53:04.067756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-29 18:53:04.067797: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-29 18:53:04.076801: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-29 18:53:04.077535: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Loading data...
Loading waveform data from: /home/ege/rawcovar_data/2020-09-09/processed_data/GAZK.npy
Waveforms shape: (687, 3000, 3)
Loading labels from: /home/ege/rawcovar_data/2020-09-09/processed_data/GAZK_y_condensed.npy
Labels shape: (687,)

Data summary:
Total samples: 687
Earthquake samples: 61
Noise samples: 626

Extracting features from trained model...
Loading model weights from: /home/ege/rawcovar/experiments/JUNE2025/29JUNE2025/1DAY_CONTINUOUS_EXP_2_2020-09/models/GAZK_representation_multiple_5epochs.h5
Using model: RepresentationLearningMultipleAutoencoder
Successfully loaded weights!
Extracting features from waveforms...
  Processed 320/687 samples...
  Processed 640/687 samples...
Extracted feature maps shape: (687, 5, 94, 64)

Found 61 earthquake samples and 626 noise samples
Will generate 10 comparison plots
Generating plot 1/10 (EQ idx: 4, Noise idx: 0)


/tmp/ipykernel_2583/1436292245.py:372: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.03, 1, 0.92])


Generating plot 2/10 (EQ idx: 22, Noise idx: 1)


/tmp/ipykernel_2583/1436292245.py:372: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.03, 1, 0.92])


...

Generating plot 9/10 (EQ idx: 64, Noise idx: 9)


/tmp/ipykernel_2583/1436292245.py:372: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.03, 1, 0.92])


Generating plot 10/10 (EQ idx: 85, Noise idx: 10)


/tmp/ipykernel_2583/1436292245.py:372: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.03, 1, 0.92])


  Saved 10 plots so far...

All plots saved to 'latent_space_plots' directory!
Generated 10 comparison plots.
